In [2]:
import torch
import torch.nn as nn
import torch.optim as optim 

import torchvision
from torchvision.datasets import CIFAR10

In [4]:
# DATASETS and DATALOADERS
from torch.utils.data import DataLoader
from torchvision.transforms import transforms


transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


train_dataset = CIFAR10(root="C:/Users/gokup/OneDrive/Desktop/PRIME/DL",download= True ,transform = transforms , train = True) 
test_dataset = CIFAR10(root="C:/Users/gokup/OneDrive/Desktop/PRIME/DL",download= True , transform= transforms , train = False)

C:\Users\gokup\AppData\Roaming\Python\Python312\site-packages\torchvision\datasets\cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


In [8]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: C:/Users/gokup/OneDrive/Desktop/PRIME/DL
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [6]:
test_dataset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: C:/Users/gokup/OneDrive/Desktop/PRIME/DL
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
# Data Loader 

train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64)

# Build the CNN

In [12]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN,self).__init__()

        # Layers
        self.conv_layer = nn.Sequential(

            # 1st Layer 
            nn.Conv2d(3,32,kernel_size=3,padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2,2), 

            # 2nd Layer 
            nn.Conv2d(32,64,kernel_size= 3, padding = 1), 
            nn.ReLU(),
            nn.MaxPool2d(2,2), 

            # 3rd Layer 
            nn.Conv2d(64,128,kernel_size=3,padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(2,2))

        self.fc_layer = nn.Sequential(
            nn.Linear(4*4*128,256), 
            nn.ReLU(),
            nn.Linear(256,10))

    def forward(self,x):
        x = self.conv_layer(x)
        x = x.view(x.size(0),-1)
        x = self.fc_layer(x)
        return x 

In [13]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train our model 

In [14]:
epochs = 10

for epoch in range(epochs):
    epoch_training_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        
        output = model.forward(images)  #FP
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()

        epoch_training_loss += loss.item()

    print(f"epoch {epoch+1}/{epochs} == loss ==  {epoch_training_loss/len(train_loader)}")

epoch 1/10 == loss ==  1.389323710937939
epoch 2/10 == loss ==  0.9758838884665838
epoch 3/10 == loss ==  0.7860757790867935
epoch 4/10 == loss ==  0.6517849708609569
epoch 5/10 == loss ==  0.5520465678876013
epoch 6/10 == loss ==  0.4516798969920334
epoch 7/10 == loss ==  0.37069823892067766
epoch 8/10 == loss ==  0.2947425155345436
epoch 9/10 == loss ==  0.2293431430369082
epoch 10/10 == loss ==  0.1905711092474058


# Eval 

In [17]:
correct_label = 0 
total_label = 0 
model.eval()

with torch.no_grad():
    for image , labels in test_loader:
        output = model.forward(image)
        _,predicted = torch.max(output,1)

        correct_label += (predicted == labels).sum().item()
        total_label+= labels.size(0)

print(total_label)
print(correct_label)
print("accuracy --",(correct_label/total_label)*100)

10000
7381
accuracy -- 73.81
